In [1]:
import pandas as pd
import numpy as np
from numba import njit
from prob_func import *
from estimators import *

# Maximum-likelihood estimation

In [8]:
def MLEstimation(NMseq,likelihood_funcs,params):
    """
    Input.
    
    1.NMseq: Sequence of triplets consisting of time (t), total active nodes (N) and total edges (M).
    2.likelihood_funcs: Sequence of joint prob. dists. Dict type i.e., parameter combination id: joint prob dist
    3.params: Sequence of parameter combinations. Dict type i.e., parameter id: (kappa, Np)
    
    Output. 
    estimates: Estimates of overall activity and population size. Type: (pandas.DataFrame)
    """
    knp_ests = []
    timeseq = []
    for elem in NMseq:
        t,N,M = elem
        prob_NM = []
        combid = []
        for fid,LF in likelihood_funcs.items():
            prob_NM.append(prob_func(N,M,LF))
            combid.append(fid)
        estparam_index = np.argmax(np.asarray(prob_NM))
        estparams_id = combid[estparam_index]
        knp_ests.append( estparams_id)
        timeseq.append(t)
        MLEresults = dict(zip(timeseq,knp_ests))
    #
    estimates = pd.DataFrame.from_dict(MLEresults,orient = "index")
    estimates.reset_index(drop = False, inplace = True)
    estimates.rename(columns = {"index":"t",0:"params"}, inplace = True)
    kappavalues, npvalues = estimators(estimates,params)
    estimates["kappa"] = kappavalues
    estimates["np"] = npvalues
    return estimates